In [1]:
from __future__ import print_function

import keras
import numpy as np

# For image display
import matplotlib.pyplot as plt
%matplotlib inline

# For switching images
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
# Load trained model and set weights
model = keras.models.load_model('../Project/model.model')
model.load_weights('../Project/model.weights')

In [3]:
path_to_labels = '/nfshome/sandbox/TrashNet-Dataset/DEMO/'
file_name = 'labels.csv'

# Load demo images and labels
demo_images = np.load('imgdat_demo.npy')
for i in range(demo_images.shape[0]):
    demo_images[i] = (demo_images[i] - demo_images[i].mean()) / demo_images[i].std()

with open(path_to_labels + file_name, 'r', encoding = 'utf-8') as f:
    demo_labels = f.read().split(',')
demo_labels = keras.utils.to_categorical(demo_labels)

print(demo_images.shape)
print(demo_labels.shape)

(50, 100, 100, 3)
(50, 6)


In [4]:
# Prep predictions and output
preds = model.predict(demo_images)
np.set_printoptions(suppress = True)

classes = ["Glass", "Paper", "Cardboard", "Plastic", "Metal", "Trash"]

# Get the index of what the network predicted
def getIndexOfHighestConfidence(pred):
    highestConfidenceIndex = 0
    for i in range(len(pred)):
        if pred[i] > pred[highestConfidenceIndex]:
            highestConfidenceIndex = i
    return highestConfidenceIndex

# Get the index of the actual classification
def getIndexOfClassification(test):
    classIndex = 0
    for i in range(len(test)):
        if test[i] == 1:
            classIndex = i
            break
    return classIndex

In [5]:
def display_prediction(index):
    global classes
    
    clear_output()
    print("Image %d of %d" % (index + 1, demo_images.shape[0]))
    print("Actual: " + classes[getIndexOfClassification(demo_labels[index])])
    print("Predicted: " + classes[getIndexOfHighestConfidence(preds[index])])
    print()
    print("Predictions: ")
    print("Glass:\t\t", np.round(preds[index, 0] * 100, 2), '%')
    print("Paper:\t\t", np.round(preds[index, 1] * 100, 2), '%')
    print("Cardboard:\t", np.round(preds[index, 2] * 100, 2), '%')
    print("Plastic:\t", np.round(preds[index, 3] * 100, 2), '%')
    print("Metal:\t\t", np.round(preds[index, 4] * 100, 2), '%')
    print("Trash:\t\t", np.round(preds[index, 5] * 100, 2), '%')

    plt.imshow(keras.preprocessing.image.array_to_img(demo_images[index, :, :, :]))
    plt.show()
    
widg_out = widgets.Output()
@widg_out.capture()
def on_button_clicked(btn):
    global img_index
    
    if btn.description == 'Previous' and img_index != 0:
        img_index = img_index - 1
        display_prediction(img_index)
    elif btn.description == 'Next' and img_index != demo_images.shape[0] - 1:
        img_index = img_index + 1
        display_prediction(img_index)
    elif btn.description == 'Start':
        display_prediction(img_index)
    
# Image to display
img_index = 0

# Set up buttons
btn_start = widgets.Button(description = "Start")
btn_start.on_click(on_button_clicked)
btn_prev = widgets.Button(description = "Previous")
btn_prev.on_click(on_button_clicked)
btn_next = widgets.Button(description = "Next")
btn_next.on_click(on_button_clicked)
display(widgets.HBox([btn_start, btn_prev, btn_next]))

display(widg_out)

Output()